In [16]:
import numpy as np
import pandas as pd
import geonamescache 
import unicodedata
import unidecode
import re
import multiprocessing as mp
import os
from multiprocessing import Pool, TimeoutError
from pandas.io.json import json_normalize

gc = geonamescache.GeonamesCache()

# read data from file
df = pd.read_csv('./data/headlines.txt', sep="\n", header=None, names=['headline'])

# get cities from geonamescache and sort them by string-length descending
cities = gc.get_cities()
known_cities=list(map(lambda k: cities[k]['name'], cities)) 
known_cities=list(map(lambda v: unidecode.unidecode(v), known_cities))
known_cities.sort(key=lambda k: len(k),reverse=True)

# get countries in same way
countries = gc.get_countries()
known_countries=list(map(lambda k: countries[k]['name'], countries))
known_countries=list(map(lambda v: unidecode.unidecode(v), known_countries))
known_countries.sort(key=lambda k: len(k),reverse=True)

# df is a working data frame
df = df.assign(cities=None)
df['cities'] =  [[] for _ in range(len(df))]
df['countries'] =  [[] for _ in range(len(df))]

print( 'number of cities = ', len(known_cities))
print( 'number of countries = ', len(known_countries))

list_of_cities=list(map(str, known_cities))
list_of_countries=list(map(str, known_countries))

city_pattern=re.compile('|'.join(map(lambda x: '\\b' + re.escape(x) + '\\b', list_of_cities)),flags=re.IGNORECASE+re.MULTILINE)
country_pattern=re.compile('|'.join(map(lambda x: '\\b' + re.escape(x) + '\\b', list_of_countries)),flags=re.IGNORECASE+re.MULTILINE)

for index, row in df.iterrows(): 
    items = re.findall( city_pattern, row['headline'] )
    for item in items:
        row['cities'].append(item)
    items = re.findall( country_pattern, row['headline'] )
    for item in items:
        row['countries'].append(item)

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
#pd.set_option('display.width', None)
#pd.set_option('display.max_colwidth', -1)

result = pd.DataFrame(columns = ['headline', 'city','country'])

for index, row in df.iterrows():
    c_headline = row['headline']
    c_cities = row['cities']
    c_countries = row['countries']
    if len(c_cities) == 0:
        c_cities.append('NaN')
    if len(c_countries) == 0:
        c_countries.append('NaN')
    for city in c_cities:
        for country in c_countries:
            new_row = { 'headline' : c_headline, 'city' : city, 'country': country }
            result = result.append( new_row, ignore_index=True)
result


number of cities =  24336
number of countries =  252


,headline,city,country
0,Zika Outbreak Hits Miami,Miami,NaN
1,Could Zika Reach New York City?,New York City,NaN
2,First Case of Zika in Miami Beach,of,NaN
3,First Case of Zika in Miami Beach,Miami Beach,NaN
4,"Mystery Virus Spreads in Recife, Brazil",Recife,Brazil
5,Dallas man comes down with case of Zika,Dallas,NaN
6,Dallas man comes down with case of Zika,man,NaN
7,Dallas man comes down with case of Zika,of,NaN
8,Trinidad confirms first Zika case,Trinidad,NaN
9,Zika Concerns are Spreading in Houston,Houston,NaN
